## 사고데이터 + 격자 병합하는 전처리과정
- 각 격자에 해당하는 사고데이터가 있는 경우 병합하여 데이터셋 구성

In [ ]:
from google.colab import drive
drive.mount('/content/Mydrive')
%cd "/content/Mydrive/MyDrive/IOT_hackathon/Github/IOT_hackathon"

In [ ]:
!pip install geopandas

import pandas as pd
import geopandas as gpd
from tqdm import tqdm
from datetime import datetime
import numpy as np

area = gpd.GeoDataFrame.from_file('data/LSMD_ADM_SECT_UMD_11.shp', encoding='cp949') # 경로 수정
grid = gpd.GeoDataFrame.from_file('Seoul_Grid/grid.shp') # (절대)경로 개별적 수정

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# datetime series for each row(grid)
grid['dt'] = 0
dt_range = pd.date_range(start="2021-10-01", end="2021-11-01", freq='H')[:-1] # from 21/01/01 to 21/12/31 by hour
for i in tqdm(range(len(grid))):
    grid.dt[i] = dt_range

  0%|          | 0/643 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
100%|██████████| 643/643 [00:00<00:00, 42639.33it/s]


In [ ]:
# Explode! (about 30s)
'''
Explode to grid(spatial) * temporal dataset(N=20244360) > about 460 MB
각 격자에 속한 시계열을 펼치는 과정
'''
grid_new = grid.explode('dt')
grid_new = grid_new.iloc[:,[1,2]]

In [ ]:
grid_new

,geometry,dt
0,"POLYGON ((935035.249 1949665.520, 935035.249 1...",2021-10-01 00:00:00
0,"POLYGON ((935035.249 1949665.520, 935035.249 1...",2021-10-01 01:00:00
0,"POLYGON ((935035.249 1949665.520, 935035.249 1...",2021-10-01 02:00:00
0,"POLYGON ((935035.249 1949665.520, 935035.249 1...",2021-10-01 03:00:00
0,"POLYGON ((935035.249 1949665.520, 935035.249 1...",2021-10-01 04:00:00
...,...,...
642,"POLYGON ((971035.249 1952665.520, 971035.249 1...",2021-10-31 19:00:00
642,"POLYGON ((971035.249 1952665.520, 971035.249 1...",2021-10-31 20:00:00
642,"POLYGON ((971035.249 1952665.520, 971035.249 1...",2021-10-31 21:00:00
642,"POLYGON ((971035.249 1952665.520, 971035.249 1...",2021-10-31 22:00:00


In [ ]:
# Merge with accident data
df_acc = gpd.read_file('Preprocessing/accidentdata.gpkg')
grid_new['accident'] = 0

for i in tqdm(range(len(df_acc))):
    idx = np.where(grid.geometry.contains(df_acc.geometry[i]))[0][0]
    dt = pd.to_datetime(df_acc.datetime[i])
    grid_new.loc[(grid_new.index == idx) & (grid_new.dt == dt), 'accident'] = 1 # 각 사고의 인덱스와 일시에 대응하는 행 추출하여 accident(사고유무) 변수를 1로 취함

grid_new.loc[grid_new.accident == 1].head(5)
    

100%|██████████| 445/445 [00:40<00:00, 10.86it/s]


,geometry,dt,accident
89,"POLYGON ((939535.249 1952165.520, 939535.249 1...",2021-11-25 19:00:00,1
114,"POLYGON ((940035.249 1952165.520, 940035.249 1...",2021-12-22 16:00:00,1
131,"POLYGON ((940535.249 1948665.520, 940535.249 1...",2021-12-30 11:00:00,1
146,"POLYGON ((941035.249 1943665.520, 941035.249 1...",2021-12-20 19:00:00,1
180,"POLYGON ((941535.249 1948665.520, 941535.249 1...",2021-10-26 14:00:00,1


In [ ]:
# 각자 파일로 저장
grid_new.to_pickle('data/accident_in_grid.pkl') # 파일 용량문제로 pickle 파일로 저장

In [ ]:
grid_new = pd.read_pickle('data/accident_in_grid.pkl')

# 전처리된 모든 데이터 병합

In [ ]:
df_env = pd.read_csv('Preprocessing/전처리완료/SDOT_Env_pros.csv') # S-DoT 환경
df_pop = pd.read_csv('Preprocessing/전처리완료/SDOT_population.csv') # S-DoT 유동인구
df_bicycle = pd.read_csv('Preprocessing/전처리완료/따릉이이용정보.csv') # 따릉이

In [ ]:
from copy import deepcopy
df_final = deepcopy(grid_new)

## 따릉이 데이터 병합

In [ ]:
df_final['count_따릉이'] = 0

In [ ]:
df_bicycle.columns

Index(['대여일자', '대여시간', '대여소번호', '대여소명', '이용건수', '이동거리', '사용시간', '성별_F', '성별_M',
       '성별_N', '연령_10대이하', '연령_20대', '연령_30대', '연령_40대', '연령_50대', '연령_60대',
       '연령_70대이상'],
      dtype='object')

In [ ]:
bicycle_loc = gpd.read_file('Preprocessing/전처리완료/따릉이.gpkg')
for i in range(len(bicycle_loc)):
    bicycle_loc.STATN_ID[i] = bicycle_loc.STATN_ID[i][3:]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
df_bicycle.head(1)

,대여일자,대여시간,대여소번호,대여소명,이용건수,이동거리,사용시간,성별_F,성별_M,성별_N,연령_10대이하,연령_20대,연령_30대,연령_40대,연령_50대,연령_60대,연령_70대이상
0,2021-09-01,0,1337,1337. 돈암성당 옆,1,2780.0,14,0,0,1,1,0,0,0,0,0,0


In [ ]:
bicycle_loc['idx_grid'] = 0
for i in range(len(bicycle_loc)):
    idx = np.where(grid.contains(bicycle_loc.geometry[i]))
    if len(idx[0]) > 0:
        bicycle_loc['idx_grid'][i] = idx[0][0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
bicycle_loc = bicycle_loc.astype({'idx_grid' : 'int'})

In [ ]:
bicycle_loc.astype()

In [ ]:
for i in tqdm(range(len(df_bicycle))):
    grid_i = bicycle_loc.loc[bicycle_loc.STATN_ID == df_bicycle['대여소번호'][i],"idx_grid"]
    datetime_i = pd.to_datetime(df_bicycle.iloc[i,:]['대여일자']).replace(hour=df_bicycle.대여시간[i])
    df_final.loc[(df_final.index == grid_i)&(df_final.dt == datetime_i),'count_따릉이'] += df_bicycle.iloc[i,4]

In [ ]:
bicycle_loc.STATN_ID

0        837
1        838
2        839
3         84
4        840
        ... 
2948    2325
2949    2326
2950    2327
2951    2328
2952    2329
Name: STATN_ID, Length: 2953, dtype: object

In [ ]:
df_bicycle['대여소번호'][0]

1337

In [ ]:
bicycle_loc.loc[bicycle_loc.STATN_ID == df_bicycle['대여소번호'][500],:]

,STATN_ID,STATN_Y,STATN_X,geometry,idx_grid


In [ ]:
bicycle_loc

,STATN_ID,STATN_Y,STATN_X,geometry,idx_grid
0,837,37.532551,126.846321,POINT (942246.029 1948338.338),203
1,838,37.493179,127.118546,POINT (966280.121 1943837.904),2128
2,839,37.499413,127.112869,POINT (965781.126 1944531.573),2098
3,84,37.580811,126.886650,POINT (945844.160 1953668.558),380
4,840,37.514088,127.099030,POINT (964564.816 1946164.790),2040
...,...,...,...,...,...
2948,2325,37.557713,127.077271,POINT (962663.642 1951013.144),1871
2949,2326,37.554539,127.043777,POINT (959703.653 1950674.838),1541
2950,2327,37.556580,127.043358,POINT (959667.744 1950901.455),1541
2951,2328,37.555611,127.043274,POINT (959659.802 1950793.986),1541


In [ ]:
df_bicycle

,대여일자,대여시간,대여소번호,대여소명,이용건수,이동거리,사용시간,성별_F,성별_M,성별_N,연령_10대이하,연령_20대,연령_30대,연령_40대,연령_50대,연령_60대,연령_70대이상
0,2021-09-01,0,1337,1337. 돈암성당 옆,1,2780.00,14,0,0,1,1,0,0,0,0,0,0
1,2021-09-01,0,1155,1155. 기쁜우리복지관,1,1420.00,13,0,0,1,0,1,0,0,0,0,0
2,2021-09-01,0,1652,1652. 파르코 앞,1,0.00,6,0,0,1,0,1,0,0,0,0,0
3,2021-09-01,0,1977,1977. 천왕역 1번 출입구 앞,1,1758.19,10,0,0,1,0,1,0,0,0,0,0
4,2021-09-01,0,1662,1662. 노원역7번출구,1,0.00,50,0,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9332161,2021-12-31,23,4561,4561. 양평역 1번출구,1,840.00,4,0,1,0,0,0,0,0,1,0,0
9332162,2021-12-31,23,815,815. LIG강촌아파트 103동앞,1,907.98,8,0,1,0,0,0,0,0,1,0,0
9332163,2021-12-31,23,2173,2173. 당곡사거리,1,2063.26,13,0,1,0,0,0,0,0,0,1,0
9332164,2021-12-31,23,2718,2718.마곡수명산파크1단지,1,950.00,8,0,1,0,0,0,0,0,0,1,0


In [ ]:
df_final.loc[df_final.index == grid_i,:]

ValueError: ignored

In [ ]:
grid_i = bicycle_loc.loc[bicycle_loc.STATN_ID == str(df_bicycle['대여소번호'][10]),"idx_grid"]
grid_i

Series([], Name: idx_grid, dtype: int64)

In [ ]:
bicycle_loc.loc[bicycle_loc['STATN_ID'] == df_bicycle['대여소번호'][10],"idx_grid"]

Series([], Name: idx_grid, dtype: int64)

In [ ]:
bicycle_loc.STATN_ID

0        837
1        838
2        839
3         84
4        840
        ... 
2948    2325
2949    2326
2950    2327
2951    2328
2952    2329
Name: STATN_ID, Length: 2953, dtype: object

In [ ]:
bicycle_loc.STATN_ID == df_bicycle['대여소번호'][10]

0       False
1       False
2       False
3       False
4       False
        ...  
2948    False
2949    False
2950    False
2951    False
2952    False
Name: STATN_ID, Length: 2953, dtype: bool

In [ ]:
따릉이_대여 = pd.read_csv('Preprocessing/전처리완료/따릉이_grid.csv')
df_final['따릉이_대여소수'] = 따릉이_대여

# 학교데이터 병합

In [ ]:
df_final[['학교_초','학교_중','학교_고','학교_대']] = 0
df_final

,geometry,dt,accident,count_따릉이,학교_초,학교_중,학교_고,학교_대,따릉이_대여소수
0,"POLYGON ((935035.249 1950165.520, 935035.249 1...",2021-09-01 00:00:00,0,0,0,0,0,0,0
0,"POLYGON ((935035.249 1950165.520, 935035.249 1...",2021-09-01 01:00:00,0,0,0,0,0,0,0
0,"POLYGON ((935035.249 1950165.520, 935035.249 1...",2021-09-01 02:00:00,0,0,0,0,0,0,0
0,"POLYGON ((935035.249 1950165.520, 935035.249 1...",2021-09-01 03:00:00,0,0,0,0,0,0,0
0,"POLYGON ((935035.249 1950165.520, 935035.249 1...",2021-09-01 04:00:00,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
2310,"POLYGON ((971535.249 1952165.520, 971535.249 1...",2021-12-31 19:00:00,0,0,0,0,0,0,0
2310,"POLYGON ((971535.249 1952165.520, 971535.249 1...",2021-12-31 20:00:00,0,0,0,0,0,0,0
2310,"POLYGON ((971535.249 1952165.520, 971535.249 1...",2021-12-31 21:00:00,0,0,0,0,0,0,0
2310,"POLYGON ((971535.249 1952165.520, 971535.249 1...",2021-12-31 22:00:00,0,0,0,0,0,0,0


In [ ]:
school = pd.read_csv('Preprocessing/전처리완료/seoul_schools_location.csv', usecols=['학교종류명','lat','long'])
univ = pd.read_csv('Preprocessing/전처리완료/seoul_univ_location.csv', encoding='cp949',usecols=['Latitude','Longitude'])

In [ ]:
gpd_school = gpd.GeoDataFrame(school,geometry = gpd.points_from_xy(x = school.long, y= school.lat, crs='WGS84'))
gpd_univ = gpd.GeoDataFrame(univ,geometry = gpd.points_from_xy(x = univ.Longitude, y= univ.Latitude, crs='WGS84'))
gpd_school = gpd_school.to_crs(epsg=5179)
gpd_univ = gpd_univ.to_crs(epsg=5179)

In [ ]:
for i in tqdm(range(len(gpd_school))):
    idx = np.where(grid.contains(gpd_school.geometry[i]))
    if len(idx[0]) > 0 :
        if gpd_school.학교종류명[i] == '초등학교':
            df_final.loc[df_final.index == idx[0][0], '학교_초'] += 1
        elif gpd_school.학교종류명[i] == '중학교':
            df_final.loc[df_final.index == idx[0][0], '학교_중'] += 1
        elif gpd_school.학교종류명[i] == '고등학교':
            df_final.loc[df_final.index == idx[0][0], '학교_고'] += 1

100%|██████████| 3852/3852 [03:08<00:00, 20.49it/s]


In [ ]:
for i in tqdm(range(len(gpd_univ))):
    idx = np.where(grid.contains(gpd_univ.geometry[i]))
    if len(idx[0]) > 0 :
        df_final.loc[df_final.index == idx[0][0], '학교_대'] += 1

100%|██████████| 64/64 [00:02<00:00, 21.51it/s]


In [ ]:
df_final.to_pickle('final.pkl')

In [ ]:
df_final = pd.read_pickle('final.pkl')

# Subway Merge

In [ ]:
# bus merging
metro = pd.read_csv('Preprocessing/전처리완료/지하철_전처리.csv')
metro_loc = pd.read_csv('Preprocessing/전처리완료/subway_location.csv')

In [ ]:
gpd_metro = gpd.GeoDataFrame(metro_loc,geometry = gpd.points_from_xy(x = metro_loc.CONV_X, y= metro_loc.CONV_Y, crs='WGS84'))
gpd_metro = gpd_metro.to_crs(epsg=5179)

In [ ]:
metro['grid'] = 0

In [ ]:
gpd_metro['idx_grid'] = 0

for i in range(len(gpd_metro)):
    idx = np.where(grid.contains(gpd_metro.geometry[i]))
    if len(idx[0]) > 0:
        gpd_metro['idx_grid'][i] = idx[0][0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
for i in tqdm(range(len(metro)//24)):
    try:
        metro.grid[i*24:(i+1)*24] = [gpd_metro.loc[gpd_metro.OUT_STN_NUM == metro.역번호[i*24],'idx_grid'] for _ in range(24)]
    except:
        TypeError

  0%|          | 0/49 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
100%|██████████| 49/49 [00:00<00:00, 123.16it/s]


In [ ]:
# metro.to_pickle('metro.pkl')

In [ ]:
metro = pd.read_pickle('metro.pkl')

In [ ]:
metro

,date,역번호,subway,grid
0,2021-09-01,150,0,965
1,2021-09-01,150,0,965
2,2021-09-01,150,0,965
3,2021-09-01,150,0,965
4,2021-09-01,150,0,965
...,...,...,...,...
821971,2021-12-31,2828,337,0
821972,2021-12-31,2828,218,0
821973,2021-12-31,2828,304,0
821974,2021-12-31,2828,192,0


In [ ]:
metro['hour'] = list(range(0,24))*(len(metro)//24)

In [ ]:
metro = metro.groupby(['date','grid','hour'], as_index = False).sum()
metro

,date,grid,hour,역번호,subway
0,2021-09-01,0,0,60228,0
1,2021-09-01,0,1,60228,0
2,2021-09-01,0,2,60228,0
3,2021-09-01,0,3,60228,0
4,2021-09-01,0,4,60228,0
...,...,...,...,...,...
696979,2021-12-31,2299,19,2562,337
696980,2021-12-31,2299,20,2562,236
696981,2021-12-31,2299,21,2562,233
696982,2021-12-31,2299,22,2562,199


In [ ]:
metro.loc[metro['grid']==0,:]

,date,grid,hour,역번호,subway
0,2021-09-01,0,0,60228,0
1,2021-09-01,0,1,60228,0
2,2021-09-01,0,2,60228,0
3,2021-09-01,0,3,60228,0
4,2021-09-01,0,4,60228,0
...,...,...,...,...,...
691291,2021-12-31,0,19,63056,21568
691292,2021-12-31,0,20,63056,15427
691293,2021-12-31,0,21,63056,16880
691294,2021-12-31,0,22,63056,10209


In [ ]:
for i in tqdm(range(len(metro)//24)):
    df_final.loc[df_final.index == metro.grid[i*24], 'subway'] = metro.loc[metro['grid']==metro.grid[i*24],'subway']

100%|██████████| 29041/29041 [15:49<00:00, 30.59it/s]


In [ ]:
df_final.to_pickle('final.pkl')

# Pub(술집)

In [ ]:
df_final = pd.read_pickle('final.pkl')

In [ ]:
pubs = pd.read_csv('Preprocessing/전처리완료/seoul_pub_location.csv', encoding='cp949', usecols = ['x','y'])
pubs.head(3)

,x,y
0,127.035690,37.501629
1,126.854370,37.555565
2,126.839973,37.568417


In [ ]:
gpd_pubs = gpd.GeoDataFrame(pubs,geometry = gpd.points_from_xy(x = pubs.x, y= pubs.y, crs='WGS84'))
gpd_pubs = gpd_pubs.to_crs(epsg=5179)

In [ ]:
gpd_pubs

,x,y,geometry
0,127.035690,37.501629,POINT (958960.405 1944808.268)
1,126.854370,37.555565,POINT (942974.726 1950886.707)
2,126.839973,37.568417,POINT (941713.136 1952321.382)
3,126.845749,37.565331,POINT (942220.792 1951975.426)
4,126.851654,37.563869,POINT (942741.188 1951809.717)
...,...,...,...
20779,127.081818,37.508396,POINT (963040.880 1945539.937)
20780,126.875847,37.524696,POINT (944848.952 1947449.181)
20781,126.970184,37.531078,POINT (953188.793 1948106.095)
20782,126.989212,37.570942,POINT (954894.004 1952519.450)


In [ ]:
df_final['pubs'] = 0

In [ ]:
for i in tqdm(range(len(gpd_pubs))):
    idx = np.where(grid.contains(gpd_pubs.geometry[i]))
    if len(idx[0]) > 0 :
        df_final.loc[df_final.index == idx[0][0], 'pubs'] += 1

100%|██████████| 20784/20784 [14:25<00:00, 24.02it/s]


In [ ]:
df_final.to_pickle('final.pkl')

# 과속방지턱

In [ ]:
df_final = pd.read_pickle('final.pkl')
speedbump = pd.read_csv('Preprocessing/전처리완료/speedbump(2020-2022).csv', encoding='cp949', usecols = ['WGS84위도','WGS84경도'])

In [ ]:
gpd_sb = gpd.GeoDataFrame(speedbump,geometry = gpd.points_from_xy(x = speedbump.WGS84경도, y= speedbump.WGS84위도, crs='WGS84'))
gpd_sb = gpd_sb.to_crs(epsg=5179)

In [ ]:
df_final['speedbump'] = 0

In [ ]:
for i in tqdm(range(len(gpd_sb))):
    idx = np.where(grid.contains(gpd_sb.geometry[i]))
    if len(idx[0]) > 0 :
        df_final.loc[df_final.index == idx[0][0], 'speedbump'] += 1

100%|██████████| 28572/28572 [20:09<00:00, 23.62it/s]


In [ ]:
df_final.to_pickle('final.pkl')

# 버스정류장

In [ ]:
df_final = pd.read_pickle('final.pkl')

In [ ]:
busstop = pd.read_csv('Preprocessing/전처리완료/busstop_location.csv', encoding='utf-8', usecols = ['CRDNT_X','CRDNT_Y'])

In [ ]:
gpd_bs = gpd.GeoDataFrame(busstop, geometry = gpd.points_from_xy(x = busstop.CRDNT_X, y= busstop.CRDNT_Y, crs='WGS84'))
gpd_bs = gpd_bs.to_crs(epsg=5179)

In [ ]:
df_final['busstop'] = 0

In [ ]:
for i in tqdm(range(len(gpd_bs))):
    idx = np.where(grid.contains(gpd_bs.geometry[i]))
    if len(idx[0]) > 0 :
        df_final.loc[df_final.index == idx[0][0], 'busstop'] += 1

100%|██████████| 69252/69252 [23:39<00:00, 48.79it/s] 


In [ ]:
df_final.to_pickle('final.pkl')